In [13]:
! pip install --upgrade ibm-watson

    100% |████████████████████████████████| 256kB 1.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for ibm-watson ... done
  Stored in directory: /home/jose/.cache/pip/wheels/b0/b4/42/15563549063278065c74f6fabcd9eae12666230edebbac9a66
  Running setup.py bdist_wheel for ibm-cloud-sdk-core ... done
  Stored in directory: /home/jose/.cache/pip/wheels/16/27/32/7d296851e2bf81e0965e313cedd4f1bc08d3eab4eaae7c133d
Successfully built ibm-watson ibm-cloud-sdk-core
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
from ibm_watson import ToneAnalyzerV3
data=pd.read_csv('processed_data/data.csv')

/home/jose/envs/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jose/envs/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
texts=data.groupby('sender').body.apply(lambda x: ' '.join(x.astype(str)))

In [26]:
from credentials import credentials
PI = PersonalityInsightsV3( 
    version="2019-05-12",
    iam_apikey=credentials_pers['apikey'],
    url=credentials_pers['url'])

In [29]:
ex=PI.tone(texts.iloc[0])

In [32]:
ex.get_result()['document_tone']['tones']

[{'score': 0.608454, 'tone_id': 'joy', 'tone_name': 'Joy'},
 {'score': 0.558697, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]

In [38]:
email_to_json={}
for index,t in texts.iteritems():
    try:
        profile=PI.profile(content=t,content_type="text/plain",accept="application/json")
        email_to_json[index]=profile
    except Exception as e:
        email_to_json[index]=None
        print(e)
    print(index)

andrea.ring@enron.com
andy.zipper@enron.com
barry.tycholiz@enron.com
benjamin.rogers@enron.com
bill.rapp@enron.com
cara.semperger@enron.com
carol.clair@enron.com
charles.weldon@enron.com
chris.dorland@enron.com
chris.germany@enron.com
cooper.richey@enron.com
dan.hyvl@enron.com
dana.davis@enron.com
danny.mccarty@enron.com
daren.j.farmer@enron.com
darrell.schoolcraft@enron.com
david.w.delainey@enron.com
debra.perlingiere@enron.com
diana.scholtes@enron.com
don.baughman@enron.com
drew.fossum@enron.com
dutch.quigley@enron.com
e..haedicke@enron.com
elizabeth.sager@enron.com
eric.bass@enron.com
errol.mclaughlin@enron.com
fletcher.j.sturm@enron.com
geoff.storey@enron.com
gerald.nemec@enron.com
greg.whalley@enron.com
holden.salisbury@enron.com
hunter.s.shively@enron.com
Error: The number of words 1 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: ee78f2f15a34001e0f1ad016414928b7
iskmgtstaceyspecial projectsStacey
j.kaminski@enron.com
jame

In [52]:
with open("processed_data/pickled_personality.pkl","wb") as file:
    pickle.dump(insights,file)

In [43]:
g=email_to_json['sally.beck@enron.com']

In [49]:
insights={i:j.get_result() for i,j in email_to_json.items() if j is not None}

In [62]:
# all are significant
personalities=pd.concat([pd.DataFrame(insight['personality'])[['percentile','name']].assign(sender=name) for name,insight in insights.items()])

In [67]:
personalities.to_csv('processed_data/personalities.csv',index=False)

In [78]:
personalities.groupby(['name','sender']).sum().unstack().percentile.T.corr()

name,Agreeableness,Conscientiousness,Emotional range,Extraversion,Openness
name,,,,,
Agreeableness,1.000000,0.453442,-0.048169,0.489249,-0.255580
Conscientiousness,0.453442,1.000000,0.626550,0.206843,0.141513
Emotional range,-0.048169,0.626550,1.000000,0.196901,0.477625
Extraversion,0.489249,0.206843,0.196901,1.000000,0.215643
Openness,-0.255580,0.141513,0.477625,0.215643,1.000000


In [33]:
# TOnes
from credentials import credentials_tone

PI = ToneAnalyzerV3(
    iam_apikey=credentials_tone["apikey"],
    url=credentials_tone["url"],
    version="2019-05-12",
)
# ex.get_result()['document_tone']['tones']

In [35]:
#Not working
tone_to_json={}
for index,t in texts.iteritems():
    try:
        profile=PI.tone(t)
        tone_to_json[index]=profile.get_result()
    except Exception, as e:
        email_to_json[index]=None
        print(e)
    print(index)

andrea.ring@enron.com
andy.zipper@enron.com
barry.tycholiz@enron.com


NameError: name 'email_to_json' is not defined